In [1]:
import OffsetArrays
using Plots
using Revise
using StatsBase
using ProgressMeter
using SparseArrays, LinearAlgebra, IndexedGraphs, Graphs
using Distributions
srcpath = "./src"
include("$srcpath/bp.jl") 
include("$srcpath/bp_stab.jl") 
include("$srcpath/post_infer.jl") 
include("$srcpath/observables.jl")

avg_ninf (generic function with 1 method)

In [2]:
λp = 1.0 # planted infection rate
λi = λp # inferred infection rate
T = 8 # discrete time: number of time-steps
γp = 0.016 # planted autoinfection probability
γi = γp # inferred autoinfection probability
N = 5000; # population size
dilution = 0.0 #dilution of observations. dil=0 means everybody observed once
fr = 0.0; #noise in the observation. 
degree_dist = Dirac(3) #distribution of the degree in the graph (Dirac = random regular).

Dirac{Int64}(value=3)

In [3]:
# Initialize model + population of messages
M_zero = ParametricModel(N=N,T=T,γp=γp,λp=λp,γi=γi,λi=λi,fr=fr,dilution=dilution,distribution=degree_dist);
#@time sweep!(M_zero)

In [4]:
# Run population dynamics (without stability analysis):
# 1. iterations 2. Free-energy
F_zero,it_zero = pop_dynamics(M_zero, tot_iterations = 40, stop_at_convergence=false);

#1.iter 2.F
0	0.21134132279813203
1	-0.22470461413573647
2	0.2362947426767074
3	0.32678866307815335
4	0.32845187537648746
5	0.3618282238601518
6	0.329624315843932
7	0.32864636986198603
8	0.3286921088697272
9	0.3276244148411759
10	0.32050690261411074
11	0.3276392798396802
12	0.3102914179797488
13	0.38245255960851066
14	0.31493585377394046
15	0.32182073049355675
16	0.40658688291428113
17	0.33150432417072734
18	0.3280861964866257
19	0.32855982518877863
20	0.32895997809894323
21	0.3236287493993441
22	0.34010553719448655
23	0.3281916887094936
24	0.32432304498593
25	0.33499919779421616
26	0.3292555347099493
27	0.3287073362459647
28	0.32885736833687695
29	0.32882295822579033
30	0.3287455144432577
31	0.3288031035437729
32	0.32533831439542266
33	0.3277738910855755
34	0.3287895771418698
35	0.32776665901078667
36	0.29047330016920603
37	0.35852756469669694
38	0.3302473676467009
39	0.3164611603597179


In [5]:
M = ParametricModel(N=N,T=T,γp=γp,λp=λp,γi=γi,λi=λi,fr=fr,dilution=dilution,distribution=degree_dist);
#@time sweep_stab!(M, 3, 4)

# Run population dynamics with stability analysis:
# 1. iterations 2. Free-energy 3. Stability parameter
pop_dynamics_stab(M, tot_iterations = 40, nbstab=20);

#1.iter 2.F 3.Δ
1	0.14503820564638287	-1
2	-0.0004142612535710793	-1
3	0.11052480201134281	-1
4	0.2426877403156737	-1
5	0.30865623011488247	-1
6	0.32579704077150584	-1
7	0.3281025077395335	-1
8	0.32860575563380723	-1
9	0.329279397315298	-1
10	0.3294641254957134	-1
11	0.3293091998254139	-1
12	0.32911465750061036	-1
13	0.3289414698958046	-1
14	0.3288169420969191	-1
15	0.32881767514474103	-1
16	0.32862384325923893	-1
17	0.3275083975787393	-1
18	0.3289120216106116	-1
19	0.3289602071226027	-1
20	0.3288521434793399	-1
21	0.3276978963270568	4.683408920398884e-7
22	0.3290120109087032	7.248009302635785e-7
23	0.32761962380914855	6.374307920933108e-7
24	0.32604037851208134	3.216905919417012e-7
25	0.3289325049165652	1.3876319907294154e-7
26	0.3274813010590629	7.999197956318135e-8
27	0.32903513105387866	4.2739993037406304e-8
28	0.3287450094683663	1.7402364642407978e-8
29	0.3287895959583609	8.490207370722055e-9
30	0.3267624356854907	1.0218197768095145e-8
31	0.3262951858870942	1.2867172630441247e-8
3